In [ ]:
!pip install pyodbc

In [ ]:
import pandas as pd
import pyodbc
from io import StringIO
from pprint import pprint
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [ ]:
# Azure Storage Account information
account_name = 'Censored'
account_key = 'Censored'
container_name = 'cis4400metrobikedata'

# Create a BlobServiceClient
blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)

# Get a reference to the container
container_client = blob_service_client.get_container_client(container_name)

# ETL process
for blob in container_client.list_blobs():
    blob_client = container_client.get_blob_client(blob.name)

    # Download the CSV content from Azure Storage
    blob_data = blob_client.download_blob()
    csv_content = blob_data.readall().decode('utf-8')

    # Transform: Perform any data transformations using pandas
    # In this example, we convert the CSV content to a pandas DataFrame
    df = pd.read_csv(StringIO(csv_content))

    # Load: Perform any data loading or further processing here
    print("Transformed DataFrame:")
    pprint(df)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
# checking for duplicate rows

duplicate_rows = df[df.duplicated()]

# Print duplicate rows, if any
if not duplicate_rows.empty:
    print("Duplicate Rows Found:")
    print(duplicate_rows)
else:
    print("No Duplicate Rows Found.")

In [ ]:
#Checking for null values

nan_counts = df.isna().sum()

# Print NaN counts for each column
print("NaN Counts for Each Column:")
print(nan_counts)

In [ ]:
df['start_lat'].fillna(0, inplace=True)
df['start_lon'].fillna(0, inplace=True)
df['end_lat'].fillna(0, inplace=True)
df['end_lon'].fillna(0, inplace=True)

In [ ]:
#Re-checking for null values

nan_counts = df.isna().sum()

# Print NaN counts for each column
print("NaN Counts for Each Column:")
print(nan_counts)

In [ ]:
# List of columns to be converted
int64_columns = df.select_dtypes(include='int64').columns

# Convert int64 columns to int32
df[int64_columns] = df[int64_columns].astype('int32')

In [ ]:
# List of columns to be converted
float64_columns = df.select_dtypes(include='float64').columns

# Convert int64 columns to int32
df[float64_columns] = df[float64_columns].astype('float32')

In [ ]:
df.dtypes

In [ ]:
# Convert all columns to lowercase
df.columns = map(str.lower, df.columns)
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

In [ ]:
# Ensure the date columns are in datetime format
date_columns = ['start_time', 'end_time']
df[date_columns] = df[date_columns].apply(pd.to_datetime, format='%m/%d/%Y %H:%M', errors='coerce')

# Extract year, month, day, hour, and minutes for 'start_time'
df['start_time_month'] = df['start_time'].dt.month
df['start_time_day'] = df['start_time'].dt.day
df['start_time_year'] = df['start_time'].dt.year
df['start_time_hour'] = df['start_time'].dt.hour
df['start_time_minutes'] = df['start_time'].dt.minute

# Extract year, month, day, hour, and minutes for 'end_time'
df['end_time_month'] = df['end_time'].dt.month
df['end_time_day'] = df['end_time'].dt.day
df['end_time_year'] = df['end_time'].dt.year
df['end_time_hour'] = df['end_time'].dt.hour
df['end_time_minutes'] = df['end_time'].dt.minute

In [ ]:
df

In [ ]:
# Generate unique station IDs based on trip IDs
df['station_id'] = df['trip_id'].astype(str) + '_' + df['start_station'].astype(str) + '_' + df['end_station'].astype(str)

# Display DataFrame
print(df[['trip_id', 'station_id']])

In [ ]:
print("Sample check:")
print(df[['trip_id', 'station_id']].sample(10))

In [ ]:
# Generating integer ID based on unique values of 'trip_id', 'start_station', 'end_station', 'bike_id', 'start_time', and 'end_time'
df['time_id'] = pd.factorize(df[['trip_id', 'start_station', 'end_station', 'bike_id', 'start_time', 'end_time']].apply(tuple, axis=1))[0] + 1

# Display the DataFrame
print(df[['trip_id', 'time_id']])

In [ ]:
print("Sample check:")
print(df[['trip_id', 'time_id']].sample(10))

In [ ]:
df.dtypes

In [ ]:
# List of columns to be converted
int64_columns = [
    'trip_id',
    'duration',
    'start_station',
    'bike_id',
    'plan_duration',
    'start_time_month',
    'start_time_day',
    'start_time_year',
    'start_time_hour',
    'start_time_minutes',
    'end_station',
    'end_time_month',
    'end_time_day',
    'end_time_year',
    'end_time_hour',
    'end_time_minutes',
    'time_id'
]

# Convert int64 columns to int32
df[int64_columns] = df[int64_columns].astype('int32')

# Display the updated DataFrame
print(df.dtypes)

In [ ]:
# List of columns to be converted to float32
float64_columns = [
    'start_lat',
    'start_lon',
    'end_lat',
    'end_lon'
]



# Convert float64 columns to float32
df[float64_columns] = df[float64_columns].astype('float32')


In [ ]:
df.dtypes

In [ ]:
df_clean = df

In [ ]:
df_clean.dtypes

In [ ]:
# Specify the path where you want to save the CSV file
save_directory = '/Users/tam/Downloads/Metro_bike_data_clean'

# Create the directory if it does not exist
os.makedirs(save_directory, exist_ok=True)

# Specify the full path including the filename
save_path = os.path.join(save_directory, 'Metro_bike_data_clean.csv')

# Save the DataFrame to CSV
df_clean.to_csv(save_path, index=False)

# Print a confirmation message
print(f"DataFrame saved to: {save_path}")